Имеются данные adult.csv (см. в материалах к занятию).

Целевой переменной является уровень дохода income (крайний правый столбец).

Описание признаков можно найти по ссылке http://www.cs.toronto.edu/~delve/data/adult/adultDetail.html

Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.

Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.

In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [2]:
data = pd.read_csv( 'adult.csv' )
data.head() # Посмотрим на данные

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
# Изучим данные 
# Посмотрим есть ли в наших признаках пустые значения (пропусков нет - это хорошо)

data.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [4]:
# Выбираем нужные признаки
selectedColumns = data[ [ 'age', 'workclass', 'educational-num', 'hours-per-week', 'income' ] ]

# столбец workclass является категориальной переменной
# переведем его в значения 0 и 1, добавив столбцы с соответствующими названиями

X = pd.get_dummies( selectedColumns, columns = [ 'workclass' ] )

# столбец income является целевой переменной, удаляем его из данных

del X['income']
X.head()

,age,educational-num,hours-per-week,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay
0,25,7,40,0,0,0,0,1,0,0,0,0
1,38,9,50,0,0,0,0,1,0,0,0,0
2,28,12,40,0,0,1,0,0,0,0,0,0
3,44,10,40,0,0,0,0,1,0,0,0,0
4,18,10,30,1,0,0,0,0,0,0,0,0


In [5]:
# целевая переменная (столбец income) является категориальной
# переведем значения столбца в числа

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit( data['income'] )
le.classes_

array(['<=50K', '>50K'], dtype=object)

In [6]:
# записываем в переменную Y преобразованный столбец income

Y = pd.Series( data = le.transform( data['income'] ) )
Y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int32

In [7]:
# Разделим данные на обучающие и тестовые

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [8]:
# Нормируем данные и Построим модель логистической регрессии

model = make_pipeline(StandardScaler(),LogisticRegression())

In [9]:
# обучаем модель

model.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [10]:
# Запускаем модель на тесте, предскажем значения целевой переменной

model.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [11]:
# Оценим качество полученной модели

model.score(X_train, Y_train)

0.7918043815262219

In [12]:
# Проверим качество полученной модели на тестовой выборке
# Чем ближе к 1, тем лучше обобщающая способность модели

model.score(X_test, Y_test) 

0.7960144680270251

In [13]:
# Нормируем данные и Используем Метод Опорных Векторов SVM

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))

In [14]:
# обучаем модель

clf.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [15]:
# Запускаем модель на тесте, предскажем значения целевой переменной

clf.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
# Оценим качество полученной модели

clf.score(X_train, Y_train)

0.8011348679399807

In [17]:
# Проверим качество полученной модели на тестовой выборке
# Чем ближе к 1, тем лучше обобщающая способность модели

clf.score(X_test, Y_test)

0.8033849723606088

In [ ]:
# Метод SVM показал немного лучшие результаты, чем логистическая регрессия